## 分別生成 html 檔案

### 垃圾桶

In [ ]:
from pymongo import MongoClient
import pandas as pd
import folium
from folium.plugins import LocateControl

# MongoDB 連接設定
client = MongoClient("mongodb://localhost:27017/")  # 連接到本地 MongoDB
db = client["DB_Final_Project"]  # 資料庫名稱
collection = db["trashcan_location"]  # 集合名稱

# 從 MongoDB 中讀取資料
data = list(collection.find({}, {"_id": 0, "address": 1, "latitude": 1, "longitude": 1}))  # 取出需要的欄位

# 將資料轉為 DataFrame
locations = []
for item in data:
    locations.append({
        "address": item["address"],
        "latitude": float(item["latitude"]),
        "longitude": float(item["longitude"])
    })

locations_df = pd.DataFrame(locations)

# 繪製地圖
if not locations_df.empty:
    # 計算地圖中心
    center_lat = locations_df["latitude"].mean()
    center_lon = locations_df["longitude"].mean()
    map_osm = folium.Map(location=[center_lat, center_lon], zoom_start=14)

    # 標記垃圾桶位置
    for _, row in locations_df.iterrows():
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            popup=f"地址: {row['address']}",
            tooltip="垃圾桶位置"
        ).add_to(map_osm)

    # 新增定位按鈕
    LocateControl().add_to(map_osm)

    # 儲存地圖為 HTML
    map_html = "trash_bins_map.html"
    map_osm.save(map_html)
    print(f"垃圾桶地圖已保存為 {map_html}，請用瀏覽器開啟查看。")
else:
    print("無有效地點資料，無法生成地圖。")


### 公園

In [ ]:
from pymongo import MongoClient
import pandas as pd
import folium
from folium.plugins import LocateControl

# MongoDB 連接設定
client = MongoClient("mongodb://localhost:27017/")  # 連接到本地 MongoDB
db = client["DB_Final_Project"]  # 資料庫名稱
collection = db["park_location"]  # 集合名稱

# 從 MongoDB 中讀取資料
data = list(collection.find({}, {"_id": 0, "Park Name": 1, "address": 1, "latitude": 1, "longitude": 1, "Opening hours": 1}))  # 取出需要的欄位

# 將資料轉為 DataFrame
locations = []
for item in data:
    locations.append({
        "park_name": item["Park Name"],
        "address": item["address"],
        "latitude": float(item["latitude"]),
        "longitude": float(item["longitude"]),
        "opening_hours": item["Opening hours"]
    })

locations_df = pd.DataFrame(locations)

# 繪製地圖
if not locations_df.empty:
    # 計算地圖中心
    center_lat = locations_df["latitude"].mean()
    center_lon = locations_df["longitude"].mean()
    map_osm = folium.Map(location=[center_lat, center_lon], zoom_start=14)

    # 標記公園位置
    for _, row in locations_df.iterrows():
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            popup=(
                f"<b>公園名稱:</b> {row['park_name']}<br>"
                f"<b>地址:</b> {row['address']}<br>"
                f"<b>營業時間:</b> {row['opening_hours']}"
            ),
            tooltip=row["park_name"]
        ).add_to(map_osm)

    # 新增定位按鈕
    LocateControl().add_to(map_osm)

    # 儲存地圖為 HTML
    map_html = "park_map.html"
    map_osm.save(map_html)
    print(f"公園地圖已保存為 {map_html}，請用瀏覽器開啟查看。")
else:
    print("無有效地點資料，無法生成地圖。")


### 動物醫院

In [ ]:
from pymongo import MongoClient
import pandas as pd
import folium
from folium.plugins import LocateControl

# MongoDB 連接設定
client = MongoClient("mongodb://localhost:27017/")  # 連接到本地 MongoDB
db = client["DB_Final_Project"]  # 資料庫名稱
collection = db["petHospital_location"]  # 集合名稱

# 從 MongoDB 中讀取資料
data = list(collection.find({}, {"_id": 0, "name": 1, "contact phone": 1, "address": 1, "latitude": 1, "longitude": 1}))  # 取出需要的欄位

# 將資料轉為 DataFrame，並處理無效資料
locations = []
for item in data:
    try:
        latitude = float(item["latitude"]) if item["latitude"] else None
        longitude = float(item["longitude"]) if item["longitude"] else None
        if latitude is not None and longitude is not None:
            locations.append({
                "name": item["name"],
                "contact_phone": item["contact phone"],
                "address": item["address"],
                "latitude": latitude,
                "longitude": longitude
            })
    except (ValueError, TypeError):
        print(f"無效資料，跳過: {item}")

# 確認資料是否存在有效記錄
if locations:
    locations_df = pd.DataFrame(locations)

    # 繪製地圖
    center_lat = locations_df["latitude"].mean()
    center_lon = locations_df["longitude"].mean()
    map_osm = folium.Map(location=[center_lat, center_lon], zoom_start=14)

    # 標記寵物醫院位置
    for _, row in locations_df.iterrows():
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            popup=(
                f"<b>醫院名稱:</b> {row['name']}<br>"
                f"<b>聯絡電話:</b> {row['contact_phone']}<br>"
                f"<b>地址:</b> {row['address']}"
            ),
            tooltip=row["name"]
        ).add_to(map_osm)

    # 新增定位按鈕
    LocateControl().add_to(map_osm)

    # 儲存地圖為 HTML
    map_html = "pet_hospital_map.html"
    map_osm.save(map_html)
    print(f"寵物醫院地圖已保存為 {map_html}，請用瀏覽器開啟查看。")
else:
    print("無有效地點資料，無法生成地圖。")


## 整合:透過按鈕切換

In [ ]:
from pymongo import MongoClient
import folium
from folium.plugins import LocateControl

# MongoDB 連接設定
client = MongoClient("mongodb://localhost:27017/")  # 連接到本地 MongoDB
db = client["DB_Final_Project"]  # 資料庫名稱

# 地圖初始化
map_osm = folium.Map(location=[25.033964, 121.564468], zoom_start=14)

# ------ 公園地圖圖層 ------
park_collection = db["park_location"]
park_data = list(park_collection.find({}, {"_id": 0, "Park Name": 1, "address": 1, "latitude": 1, "longitude": 1, "Opening hours": 1}))

park_layer = folium.FeatureGroup(name="公園地圖")
for item in park_data:
    try:
        folium.Marker(
            location=[float(item["latitude"]), float(item["longitude"])],
            popup=(
                f"<b>公園名稱:</b> {item['Park Name']}<br>"
                f"<b>地址:</b> {item['address']}<br>"
                f"<b>營業時間:</b> {item['Opening hours']}"
            ),
            tooltip=item["Park Name"]
        ).add_to(park_layer)
    except (ValueError, TypeError):
        print(f"跳過無效公園資料: {item}")

park_layer.add_to(map_osm)

# ------ 寵物醫院地圖圖層 ------
hospital_collection = db["petHospital_location"]
hospital_data = list(hospital_collection.find({}, {"_id": 0, "name": 1, "contact phone": 1, "address": 1, "latitude": 1, "longitude": 1}))

hospital_layer = folium.FeatureGroup(name="寵物醫院地圖")
for item in hospital_data:
    try:
        folium.Marker(
            location=[float(item["latitude"]), float(item["longitude"])],
            popup=(
                f"<b>醫院名稱:</b> {item['name']}<br>"
                f"<b>聯絡電話:</b> {item['contact phone']}<br>"
                f"<b>地址:</b> {item['address']}"
            ),
            tooltip=item["name"]
        ).add_to(hospital_layer)
    except (ValueError, TypeError):
        print(f"跳過無效寵物醫院資料: {item}")

hospital_layer.add_to(map_osm)

# ------ 垃圾桶地圖圖層 ------
trashcan_collection = db["trashcan_location"]
trashcan_data = list(trashcan_collection.find({}, {"_id": 0, "address": 1, "latitude": 1, "longitude": 1}))

trashcan_layer = folium.FeatureGroup(name="垃圾桶地圖")
for item in trashcan_data:
    try:
        folium.Marker(
            location=[float(item["latitude"]), float(item["longitude"])],
            popup=f"地址: {item['address']}",
            tooltip="垃圾桶位置"
        ).add_to(trashcan_layer)
    except (ValueError, TypeError):
        print(f"跳過無效垃圾桶資料: {item}")

trashcan_layer.add_to(map_osm)

# ------ 新增圖層控制按鈕 ------
folium.LayerControl().add_to(map_osm)

# ------ 新增定位按鈕 ------
LocateControl().add_to(map_osm)

# 儲存地圖為 HTML
map_html = "multi_layer_map.html"
map_osm.save(map_html)
print(f"多功能地圖已保存為 {map_html}，請用瀏覽器開啟查看。")
